In [84]:
import numpy as np 
import pandas as pd

from berkeley_pes.utils.data import *

ValueError: attempted relative import beyond top-level package

In [54]:
file_rapter = "../../data/20230414_rapter_tracks_initial.json"
file_libe = "../../data/tasks_opt_trajectories_partial.json"
file_test = "../../data/test_rapter.json"

df_rapter = pd.read_json(file_test)
#print(df_rapter.shape)
#df_libe = dd.read_json(file_libe)
#pd.read_json(file_libe)

In [14]:
#write_to_npz(res, file_out_npz)
#write_to_ase(res, file_out_ase)

       
write_whole_df_to_ase_xyz(
    "../../data/tasks_opt_trajectories_partial.json", 
    "../../data/npz/opt_trajectories_partial.xyz")

done loading json


0it [00:01, ?it/s]


KeyboardInterrupt: 

In [78]:
import ijson
import numpy as np 

def parse_json(json_filename, mode="normal"):

    energies_raw, energies = [], []
    grad_unformated = []
    xyz_unformated = []
    element_list = []
    atom_count = []
    element_count = []

    with open(json_filename, 'rb') as input_file:
        # load json iteratively
        parser = ijson.parse(input_file)
        ind_track, atom_count_temp = 0, 0
        ind_current = -1
        ind_mode = -1
        check_ind = 0
        trigger_count = 0 
        for prefix, event, value in parser:
            check_ind +=1
            if check_ind % 10000000 == 0:
                print('prefix={}, event={}, value={}'.format(prefix, event, value))

            if mode=="alt": 
                #pass
                
                if prefix[0:13] == "item.molecule":
                    if ind_mode == -1 and event == "start_array":
                        try:
                            ind_current = int(prefix.split(".")[1])
                            ind_mode = 0
                            print("entering ind mode 0")
                        except: 
                            trigger = False
                            ind_mode = 1    
                            print("entering ind mode 1")
                    
                    if value == None and event == "end_map" and prefix=="item.molecule":
                        trigger = True
                        trigger_count +=1

                    elif event == 'number':
                        if "xyz" in prefix.split("."):
                            #print('prefix={}, event={}, value={}'.format(prefix, event, value))
                            xyz_unformated.append(float(value))

                    elif event == 'string': 
                        #print('prefix={}, event={}, value={}'.format(prefix, event, value))
                        if "element" in prefix.split("."):
                            #print('prefix={}, event={}, value={}'.format(prefix, event, value))
                            element_list.append(str(value))
                            #print(value)
                            if trigger:
                                #print("triggered")
                                atom_count.append(atom_count_temp)
                                element_count.append(len(set(element_list)))
                                atom_count_temp = 1
                                trigger = False

                            else: 
                                atom_count_temp += 1
                
                if value!=None:
                    if "gradient" in prefix.split("."):
                        if event == 'number':
                            grad_unformated.append(float(value))
                
                    if prefix=='item.energy':
                        if event == 'number':
                            energies_raw.append(float(value))
                
            else: 
                if value is not None:
                    if event == "string":
                        if "formula_alphabetical" in prefix.split("."): 
                            # sum all integers in string
                            elements = value.split()
                            # strip elements of alphabetical characters
                            elements = [element.strip("ABCDEFGHIJKLMNOPQRSTUVWXYZ") for element in elements]
                            #strip lowercase letters
                            elements = [element.strip("abcdefghijklmnopqrstuvwxyz") for element in elements]
                            #print(elements)
                            num_atoms = sum(map(int, elements))
                            element_count.append(num_atoms)

                    if event == 'number':
                        if "energy_trajectory" in prefix.split("."):
                            energies_raw.append(float(value))

                    if event == 'number':
                        if "gradient_trajectory" in prefix.split("."):
                            #print('prefix={}, event={}, value={}'.format(prefix, event, value))
                            grad_unformated.append(float(value))

                if "molecule_trajectory" in prefix.split("."):
                    #if prefix[0:13] == "item.molecule":                    
                    if ind_mode == -1 and event == "start_array":
                        try:
                            ind_current = int(prefix.split(".")[1])
                            ind_mode = 0
                            print("entering ind mode 0")
                        except: 
                            trigger = False
                            ind_mode = 1    
                            print("entering ind mode 1")
                                        
                    if ind_mode == 0:
                        #print('prefix={}, event={}, value={}'.format(prefix, event, value))

                        if event == 'number':
                            if "xyz" in prefix.split("."):
                                #print('prefix={}, event={}, value={}'.format(prefix, event, value))
                                xyz_unformated.append(float(value))

                        if event == 'string': 
                            #print('prefix={}, event={}, value={}'.format(prefix, event, value))
                            if "name" in prefix.split("."):
                                element_list.append(str(value))
                                ind_current = int(prefix.split(".")[1])
                                #print(ind_current, ind_track)
                                if ind_current != ind_track:
                                    ind_track = int(ind_current)
                                    atom_count.append(atom_count_temp)
                                    atom_count_temp = 1
                                else: 
                                    atom_count_temp += 1
                
                    if ind_mode == 1:
                        #print('prefix={}, event={}, value={}'.format(prefix, event, value))
                        if value == None and event == "end_array" and prefix=="item.molecule_trajectory":
                            trigger = True
                            trigger_count +=1

                        if event == 'number':
                            if "xyz" in prefix.split("."):
                                xyz_unformated.append(float(value))

                        if event == 'string': 
                            if "name" in prefix.split("."):
                                element_list.append(str(value))
                                if trigger:
                                    atom_count.append(atom_count_temp)
                                    atom_count_temp = 1
                                    trigger = False
                                else: 
                                    atom_count_temp += 1

    if ind_mode == 0:                
        atom_count.append(atom_count_temp)
    else:
        atom_count.append(atom_count_temp)

    print(trigger_count) 
    print(len(energies_raw))
    print(len(grad_unformated))
    print(len(xyz_unformated))
    print(len(element_list))
    print(len(atom_count)) #
    print(len(element_count)) 

    grad_unformated = np.array(grad_unformated)
    grad_formated = grad_unformated.reshape(-1, 3)
    xyz_unformated = np.array(xyz_unformated)
    xyz_formated = xyz_unformated.reshape(-1, 3)
    atom_count = np.array(atom_count)
    frames_per_mol = atom_count / element_count
    
    #print(frames_per_mol)
    grad_format = np.split(grad_formated, np.cumsum(atom_count)[:-1])
    xyz_format = np.split(xyz_formated, np.cumsum(atom_count)[:-1])
    element_list = np.split(element_list, np.cumsum(atom_count)[:-1]) #
    
    #energies = np.split(energies, np.cumsum(frames_per_mol)[:-1])
    # split energies into frames per molecule
    running_start = 0
    for i in range(len(frames_per_mol)):
        energies.append(energies_raw[running_start:running_start+int(frames_per_mol[i])])
        #energies.append(energies_raw[i*int(frames_per_mol[i]):(i+1)*int(frames_per_mol[i])])
        running_start += int(frames_per_mol[i])
    xyz_format = [array.reshape(int(frames_per_mol[ind_frame]), element_count[ind_frame], 3) for ind_frame, array in enumerate(xyz_format)]
    grad_format = [array.reshape(int(frames_per_mol[ind_frame]), element_count[ind_frame], 3) for ind_frame, array in enumerate(grad_format)]
    element_list = [array.reshape(int(frames_per_mol[ind_frame]), element_count[ind_frame]) for ind_frame, array in enumerate(element_list)]

    data = {
        "energies": energies,
        "grads": grad_format,
        "xyz": xyz_format,
        "elements": element_list,
        "frames_per_mol": frames_per_mol,
        "atom_count": atom_count
    }
    return data

if __name__ == '__main__':
    # running
    data = parse_json('../../data/20230414_rapter_tracks_initial.json')
    #data = parse_json('../../data/tasks_opt_trajectories_partial.json', mode="alt")
    data_test = parse_json('../../data/test_rapter.json') # works



entering ind mode 1
prefix=item.molecule_trajectory.item.sites.item, event=map_key, value=xyz
prefix=item.molecule_trajectory.item.sites.item.species.item, event=start_map, value=None
prefix=item.molecule_trajectory.item.sites.item.species.item.element, event=string, value=H
prefix=item.molecule_trajectory.item.sites.item.species.item, event=end_map, value=None
prefix=item.molecule_trajectory.item.sites.item.species.item.occu, event=number, value=1
prefix=item.gradient_trajectory.item.item, event=start_array, value=None
prefix=item.molecule_trajectory.item.sites.item.xyz, event=start_array, value=None
prefix=item.molecule_trajectory.item.sites.item.species.item, event=start_map, value=None
prefix=item.gradient_trajectory.item.item, event=end_array, value=None
prefix=item.molecule_trajectory.item.sites.item.properties, event=end_map, value=None
prefix=item.molecule_trajectory.item.sites.item, event=map_key, value=name
prefix=item.molecule_trajectory.item.sites.item.xyz.item, event=numbe

100

In [75]:


def write_whole_df_to_ase_xyz(dict_info, file_out): 
    """
    Takes a dataframe and writes it to a npz file
    """
    energies = dict_info["energies"]
    grads_list = dict_info["grads"]
    xyzs_list = dict_info["xyz"]
    elements_list = dict_info["elements"]
    frame_count_global = 0 
    with open(file_out, "w") as f:    
        for ind_frame, (energies_frame, grads_frame, xyzs_frame, elements_frame) in enumerate(zip(energies, grads_list, xyzs_list, elements_list)):
            #print(ind_frame, len(energies_frame))
            for ind_mol, (energy, grad, xyz, elements) in enumerate(zip(energies_frame, grads_frame, xyzs_frame, elements_frame)):
                frame_count_global += 1
                n_atoms = len(elements)
                #print(elements)
                f.write(str(n_atoms) + "\n")
                f.write("Properties=species:S:1:pos:R:3:forces:R:3 energy={} free_energy={} pbc=\"F F F\"\n".format(energy, energy))
                for ind_atom, (xyz, grad) in enumerate(zip(xyz, grad)): 
                    #print(elements[0])
                    f.write("{:2} {:15.8} {:15.8} {:15.8} {:15.8} {:15.8} {:15.8}\n".format(elements[ind_atom], xyz[0], xyz[1], xyz[2], grad[0], grad[1], grad[2]))

    
    print("Wrote {} frames to {}".format(frame_count_global, file_out))


In [76]:
frame_count = 0 
for i in range(len(data_test["energies"])):
    frame_count += len(data_test["energies"][i])
print(frame_count)


2755


In [81]:
write_whole_df_to_ase_xyz(data_test, "../../data/npz/test_parser.xyz")
write_whole_df_to_ase_xyz(data, "../../data/npz/20230414_rapter_tracks_initial.xyz")

Wrote 2755 frames to ../../data/npz/test_parser.xyz
Wrote 579636 frames to ../../data/npz/20230414_rapter_tracks_initial.xyz


In [82]:
# read with ase 
from ase.io import read
test_out = read("../../data/npz/20230414_rapter_tracks_initial.xyz", index=":")

In [83]:
len(test_out) #whoo!!

579636